# Real-world dataset pre-processing

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import pandas as pd

import pickle
from modules import helpers
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer, IterativeImputer

helpers.seed(1)

In [ ]:
full_df = pd.read_table('../data/prison/prison_data.tsv')

In [ ]:
drug_keys = ["V0062", "V0064", "V0066", "V0068", "V0098", "V0100", "V0102", "V0124", "V0125", "V0126", "V0127", "V0128", "V0150", "V0151", "V0152", "V0153","V0154", 
             "V0178", "V0179", "V0180", "V0181","V0182", "V0204", "V0205", "V0206", "V0207", "V0208", "V0230", "V0231", "V0232", "V0233", "V0234", "V0256", "V0257", 
             "V0258", "V0259", "V0260", "V0280", "V0281", "V0282", "V0283", "V0284", "V0306", "V0307", "V0308", "V0332", "V0333", "V0334", "V0335", "V0336", "V0357", 
             "V0358", "V0359", "V0360","V0380", "V0381", "V0382", "V0383", "V0384", "V0477", "RV0036", "RV0037"]
drug_keys_3 = ["V0062", "V0098", "V0100", "V0102", "V0124", "V0125", "V0126", "V0127", "V0128", "V0150", "V0151", "V0152", "V0153","V0154", 
             "V0178", "V0179", "V0180", "V0181","V0182", "V0204", "V0205", "V0206", "V0207", "V0208", "V0230", "V0231", "V0232", "V0233", "V0234", "V0256", "V0257", 
             "V0258", "V0259", "V0260", "V0280", "V0281", "V0282", "V0283", "V0284", "V0306", "V0307", "V0308", "V0332", "V0333", "V0334", "V0335", "V0336", "V0357", 
             "V0358", "V0359", "V0360","V0380", "V0381", "V0382", "V0383", "V0384", "RV0037"]
# drug: all==3; besides (V0064, V0066, V0068, V0477) == 4; RV0036 == 8,9,10, RV0037 == 3 -> overall controlling variable

offense_type_keys = ["V0675", "V0761", "V0762", "V0763", "V0764", "V0765", "V0766", "V0767", "V0778", "V0885", "V0886", "V0888"]
# drop:  -9,-8,-2,-1
# possibly include type of drug: V0677 - V0687
# amount of drug: V0721 - V0740


crime_history_keys = ["V0899", "V0900", "V0902", "V0903", "V0905", "V0906", "V0908", "V0909", "V0913", "V0918"]
# drop:  -9,-8,-2,-1
#combine "V0913" &"V0918" to one variable (age of first admission)

family_keys = ["V1172", "V1173", "V1174", "V1175", "V1176"]
# drop:  -9,-8,-2,-1

birthplace_keys =["V0951", "RV0004"]
#drop: RV0004: 8
# RV0004: US citizen: 1:yes, 2: no; 
# V0951 # years lived in US (numerical) -> drop -1,-2,-8,-9


race_keys = ["V0015","V0016","V0018","V0019","V0021"]
# Yes = 1, No = 0

sentence_keys = ["V0396", "V0397", "V0398", "V0399", "V0401", "V0402", "V0403", "V0404", "V0405", "V0406", "V0407", "V0408", "V0409", "V0410", "V0411", "V0412", "V0413", 
                 "V0414", "V0415", "V0417", "V0418", "V0419", "V0420", "V0421", "V0422", "V0453", "V0454", "V0455", "V0458"]
#drop: -9,-8,-2,-1
# maybe exclude V0453 - V0455, V0458

all_keys = drug_keys + offense_type_keys + crime_history_keys + housing_keys + family_keys + birthplace_keys + race_keys + sentence_keys
drug_df = full_df.loc[full_df["RV0037"]==3][all_keys]

In [ ]:
# encode race
# white = 1, non-white = 0
drug_df["race"] = drug_df["V0016"].replace("2",0).astype(int)
drug_df["race"] = drug_df["race"].replace(2,0).astype(int)
drug_df = drug_df.drop(columns=race_keys)

In [ ]:
# encode offense type

drug_df = drug_df.loc[drug_df["RV0036"] != "10"]      # delete offense type "other drug" 
drug_df = drug_df.loc[drug_df["RV0036"] != 10]
drug_df["RV0036"] = drug_df["RV0036"].replace("8",0)  # -> 0 = drug trafficking
drug_df["RV0036"] = drug_df["RV0036"].replace(8,0) 
drug_df["RV0036"] = drug_df["RV0036"].replace("9",1)  # -> 1 = drug possession
drug_df["RV0036"] = drug_df["RV0036"].replace(9,1)
drug_df["offense_type_general"] = drug_df["RV0036"].astype(int)

# drug_df["V0675"].unique() -> drug possenssion 1 yes, 2 no

#import drug to US
drug_df = drug_df.loc[drug_df["V0761"] != "-2"] #drop refusal
drug_df = drug_df.loc[drug_df["V0761"] != "-1"] # drop "don't knoW"
drug_df = drug_df.loc[drug_df["V0761"] != " "] # drop 
drug_df["import"] = drug_df["V0761"].replace("2","0").astype(int) # yes:1, no:0

# growing/manufacturing drug
drug_df = drug_df.loc[drug_df["V0762"] != "-1"] # drop "don't know"
drug_df["grow"] = drug_df["V0762"].replace("2","0").astype(int) # yes:1, no:0

# Laundring drug money
drug_df = drug_df.loc[drug_df["V0763"] != "-1"] # drop "don't know"
drug_df = drug_df.loc[drug_df["V0763"] != "-2"] # drop refusal
drug_df["money laundry"] = drug_df["V0763"].replace("2","0").astype(int) # yes:1, no:0

# Usage/posession
drug_df = drug_df.loc[drug_df["V0766"] != "-2"] # drop refusal
drug_df["possession"] = drug_df["V0766"].replace("2","0").astype(int) # yes:1, no:0

# Selling
drug_df["selling"] = ((drug_df["V0764"]=="1").astype(int) + (drug_df["V0765"]=="1").astype(int)).replace(2,1)
# maybe drop more individuals

drug_df = drug_df.drop(columns = offense_type_keys)

In [ ]:
# encode birthplace

drug_df = drug_df.loc[drug_df["RV0004"] != "8"]  # -> drop all individuals with missing information on US citizenship status 
drug_df = drug_df.loc[drug_df["RV0004"] != 8]
drug_df = drug_df.loc[drug_df["RV0004"] != " "]
drug_df["citizenship"] = drug_df["RV0004"].astype(int).replace(2,0) # -> non US-citizenship = 0, citizen = 1

drug_df = drug_df.drop(columns=birthplace_keys)

In [ ]:
# encode familiy background (= familiy members sentenced before)
#1: yes, 0: no

df_family = drug_df[family_keys]
fam = df_family.isin([1]).any(axis=1).astype(int)
drug_df["family_history"] = fam
drug_df = drug_df.drop(columns=family_keys)

In [ ]:
# enocde crime history

# number of prison sentences in crime history
drug_df = drug_df.loc[drug_df["V0909"]!=" "]
drug_df = drug_df.loc[drug_df["V0909"].astype(int) != -2]
drug_df = drug_df.loc[drug_df["V0909"].astype(int) != -1]
drug_df["n_sentences_history"] = drug_df["V0909"].astype(int)
for n in drug_df["n_sentences_history"].unique():
    if n >= 10:
        drug_df["n_sentences_history"] = drug_df["n_sentences_history"].replace(n, 3)
    elif (n >= 5) and (n<=9):
        drug_df["n_sentences_history"] = drug_df["n_sentences_history"].replace(n, 2)
    elif (n>=1) and (n<=4):
        drug_df["n_sentences_history"] = drug_df["n_sentences_history"].replace(n, 1)       

# age of first admission to prison

drug_df = drug_df.drop(columns = crime_history_keys)

In [ ]:
# encode sentence 
drug_df["sentence_years"] = drug_df["V0396"].loc[drug_df["V0396"] != -1]
drug_df["sentence_years"] = drug_df["sentence_years"].replace(' ', np.nan)
drug_df.loc[drug_df["sentence_years"]!= np.nan,"sentence_years"]  = drug_df.loc[drug_df["sentence_years"]!= np.nan,"sentence_years"].astype(float)

drug_df["sentence_months"] = drug_df["V0397"]
drug_df["sentence_months"] = drug_df["sentence_months"].replace(' ', np.nan)
drug_df.loc[drug_df["sentence_months"]!= np.nan,"sentence_months"]  = drug_df.loc[drug_df["sentence_months"]!= np.nan,"sentence_months"].astype(float)

drug_df["sentence_weeks"] = drug_df["V0398"]
drug_df["sentence_weeks"] = drug_df["sentence_weeks"].replace(' ', np.nan)
drug_df.loc[drug_df["sentence_weeks"]!= np.nan,"sentence_weeks"]  = drug_df.loc[drug_df["sentence_weeks"]!= np.nan,"sentence_weeks"].astype(float)

drug_df["sentence_days"] = drug_df["V0399"]
drug_df["sentence_days"] = drug_df["sentence_days"].replace(' ', np.nan)
drug_df.loc[drug_df["sentence_days"]!= np.nan,"sentence_days"]  = drug_df.loc[drug_df["sentence_days"]!= np.nan,"sentence_days"].astype(float)

In [ ]:
drug_df = drug_df.drop(columns = drug_keys)
drug_df = drug_df.drop(columns = sentence_keys)

In [ ]:
drug_df["sentence"] = drug_df["sentence_months"]

drug_df = drug_df.drop(columns = ["sentence_years", "sentence_months", "sentence_weeks", "sentence_days"])

In [ ]:
imputer = KNNImputer(n_neighbors=10)
drug_df_nonwhite = pd.DataFrame(imputer.fit_transform(drug_df.loc[drug_df["race"]==0.0].values.astype(float)), columns = drug_df.keys())
drug_df_white = pd.DataFrame(imputer.fit_transform(drug_df.loc[drug_df["race"]==1.0].values.astype(float)), columns = drug_df.keys())

In [ ]:
drug_df = pd.concat([drug_df_white, drug_df_nonwhite], axis = 0)

In [ ]:
drug_df_white = drug_df_white.loc[drug_df_white["sentence"] >= 0]
drug_df_nonwhite = drug_df_nonwhite.loc[drug_df_nonwhite["sentence"] >= 0]

In [ ]:
drug_df = pd.concat([drug_df_white, drug_df_nonwhite], axis = 0)
drug_df = drug_df.sample(frac=1)

In [ ]:
drug_df = drug_df.rename(columns={"race": "A", "sentence": "Y", "n_sentences_history": "M"})
drug_df = drug_df.rename(columns={"offense_type_general": "Z1", "import": "Z2", "grow": "Z3", "money laundry": "Z4", "possession":"Z5", "selling":"Z6", "family_history":"UIE", "citizenship":"UDE"})
drug_df["USE"] = np.empty(len(drug_df["UIE"]))

In [ ]:
drug_df = drug_df.loc[drug_df["Y"]<=12]

In [ ]:
drug_df_white = drug_df.loc[drug_df["A"]==1.0]
drug_df_nonwhite = drug_df.loc[drug_df["A"]==0.0]

In [ ]:
with open("../data/prison/prison_dataframe", "wb") as output_file:
    pickle.dump(drug_df, output_file)